In [7]:
## download packages
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shi.__.ling\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shi.__.ling\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shi.__.ling\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
## import libraries
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [9]:
## read the data
df_raw = pd.read_csv('../Data/Raw/Job_Frauds.csv', encoding='latin-1')

In [10]:
# inspect the data
df = df_raw
print(df.info())   # Check data types & missing values
print(df.head())   # View first few rows
print(df['Fraudulent'].value_counts())  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Job Title           17880 non-null  object
 1   Job Location        17534 non-null  object
 2   Department          6333 non-null   object
 3   Range_of_Salary     2868 non-null   object
 4   Profile             14572 non-null  object
 5   Job_Description     17879 non-null  object
 6   Requirements        15185 non-null  object
 7   Job_Benefits        10670 non-null  object
 8   Telecomunication    17880 non-null  int64 
 9   Comnpany_Logo       17880 non-null  int64 
 10  Type_of_Employment  14409 non-null  object
 11  Experience          10830 non-null  object
 12  Qualification       9775 non-null   object
 13  Type_of_Industry    12977 non-null  object
 14  Operations          11425 non-null  object
 15  Fraudulent          17880 non-null  int64 
dtypes: int64(3), object(13

In [11]:
# check for missing values
print(df.isnull().sum())

Job Title                 0
Job Location            346
Department            11547
Range_of_Salary       15012
Profile                3308
Job_Description           1
Requirements           2695
Job_Benefits           7210
Telecomunication          0
Comnpany_Logo             0
Type_of_Employment     3471
Experience             7050
Qualification          8105
Type_of_Industry       4903
Operations             6455
Fraudulent                0
dtype: int64


In [12]:
# check if missing values correlates with whether job is fraudulent
print(df[df['Department'].isnull()]['Fraudulent'].value_counts())
print(df[df['Range_of_Salary'].isnull()]['Fraudulent'].value_counts())
print(df[df['Profile'].isnull()]['Fraudulent'].value_counts())
print(df[df['Requirements'].isnull()]['Fraudulent'].value_counts())
print(df[df['Job_Benefits'].isnull()]['Fraudulent'].value_counts())
print(df[df['Type_of_Employment'].isnull()]['Fraudulent'].value_counts())
print(df[df['Experience'].isnull()]['Fraudulent'].value_counts())
print(df[df['Qualification'].isnull()]['Fraudulent'].value_counts())
print(df[df['Type_of_Industry'].isnull()]['Fraudulent'].value_counts())
print(df[df['Operations'].isnull()]['Fraudulent'].value_counts())


0    11016
1      531
Name: Fraudulent, dtype: int64
0    14369
1      643
Name: Fraudulent, dtype: int64
0    2721
1     587
Name: Fraudulent, dtype: int64
0    2541
1     154
Name: Fraudulent, dtype: int64
0    6846
1     364
Name: Fraudulent, dtype: int64
0    3230
1     241
Name: Fraudulent, dtype: int64
0    6615
1     435
Name: Fraudulent, dtype: int64
0    7654
1     451
Name: Fraudulent, dtype: int64
0    4628
1     275
Name: Fraudulent, dtype: int64
0    6118
1     337
Name: Fraudulent, dtype: int64


In [13]:
# We can't simply drop missing vallues as they take up quite a significant portion of Fraudulant cases. 
# The presence of missing values may correlate with whether a job posting is fraudulent
# Have to check correlation when doing EDA/feature engineering

In [14]:
## split 'Job Location' into 'Country', 'State', 'City'
split_location = df['Job Location'].str.split(', ', expand=True)
df['Country'] = split_location[0]  
df['State'] = split_location[1] 
df['City'] = split_location[2] 
df = df.drop(columns=['Job Location'])

In [15]:
text_cols = ['Job Title','Profile', 'Job_Description', 'Requirements', 'Job_Benefits','Type_of_Employment','Experience','Qualification',
        'Department', 'Type_of_Industry', 'Operations','Country','State','City']
df[text_cols] = df[text_cols].fillna('unknown')

In [16]:
df.head()

,Job Title,Department,Range_of_Salary,Profile,Job_Description,Requirements,Job_Benefits,Telecomunication,Comnpany_Logo,Type_of_Employment,Experience,Qualification,Type_of_Industry,Operations,Fraudulent,Country,State,City
0,Marketing Intern,Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,unknown,0,1,Other,Internship,unknown,unknown,Marketing,0,US,NY,New York
1,Customer Service - Cloud Video Production,Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,Full-time,Not Applicable,unknown,Marketing and Advertising,Customer Service,0,NZ,,Auckland
2,Commissioning Machinery Assistant (CMA),unknown,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,unknown,0,1,unknown,unknown,unknown,unknown,unknown,0,US,IA,Wever
3,Account Executive - Washington DC,Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI â Environmental Systems Re...,"EDUCATION:Â Bachelorâs or Masterâs in GIS,...",Our culture is anything but corporateâwe hav...,0,1,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,US,DC,Washington
4,Bill Review Manager,unknown,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,US,FL,Fort Worth


In [17]:
## convert strings to lower case
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [18]:
## remove punctuations and symbols
df = df.replace(to_replace=r'[^\w\s]', value='', regex=True)

In [19]:
## tokenize, remove stop words, and lemmatize for columns 'Profile', 'Job_Description', 'Requirements', 'Job_Benefits','Job Title', 'Department', 'Type_of_Industry', 'Operations'
# initialize stop words and lemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()    
cols = ['Profile', 'Job_Description', 'Requirements', 'Job_Benefits','Job Title', 'Department', 'Type_of_Industry', 'Operations']

In [20]:
for col in cols:
    # apply tokenization 
    df[col] = df[col].apply(lambda x: word_tokenize(x) if isinstance(x, str) and x.strip() != '' else [])
    # remove stop words
    df[col] = df[col].apply(lambda x: [word for word in x if word not in stop_words])
    # apply lemmatization
    df[col] = df[col].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    
print(df.head())

                                       Job Title   Department Range_of_Salary  \
0                            [marketing, intern]  [marketing]             NaN   
1  [customer, service, cloud, video, production]    [success]             NaN   
2     [commissioning, machinery, assistant, cma]    [unknown]             NaN   
3           [account, executive, washington, dc]       [sale]             NaN   
4                        [bill, review, manager]    [unknown]             NaN   

                                             Profile  \
0  [food52, weve, created, groundbreaking, awardw...   
1  [90, second, world, cloud, video, production, ...   
2  [valor, service, provides, workforce, solution...   
3  [passion, improving, quality, life, geography,...   
4  [spotsource, solution, llc, global, human, cap...   

                                     Job_Description  \
0  [food52, fastgrowing, james, beard, awardwinni...   
1  [organised, focused, vibrant, awesomedo, passi...   
2  [clie

In [21]:
for col in cols:
    df[col] = df[col].apply(lambda x: " ".join(x))
print(df.head())

                                 Job Title Department Range_of_Salary  \
0                         marketing intern  marketing             NaN   
1  customer service cloud video production    success             NaN   
2    commissioning machinery assistant cma    unknown             NaN   
3          account executive washington dc       sale             NaN   
4                      bill review manager    unknown             NaN   

                                             Profile  \
0  food52 weve created groundbreaking awardwinnin...   
1  90 second world cloud video production service...   
2  valor service provides workforce solution meet...   
3  passion improving quality life geography heart...   
4  spotsource solution llc global human capital m...   

                                     Job_Description  \
0  food52 fastgrowing james beard awardwinning on...   
1  organised focused vibrant awesomedo passion cu...   
2  client located houston actively seeking experi...   


In [22]:
# check if data is imbalanced
print(df['Fraudulent'].value_counts())

0    17014
1      866
Name: Fraudulent, dtype: int64


In [23]:
# conduct undersampling since data is imbalanced
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(df.drop(columns=['Fraudulent']), df['Fraudulent'])
df_uresampled = pd.concat([X_resampled, y_resampled], axis=1)
print(df_uresampled)

                                Job Title      Department Range_of_Salary  \
13092       sr m dynamic consultant ax gp         unknown             NaN   
13436           software engineer android         unknown             NaN   
17502              twic rapidgate laborer         unknown             NaN   
10820          agentinbound sale position         unknown        11501150   
2067         internship medium production         unknown        45004500   
...                                   ...             ...             ...   
17827  student position parttime fulltime         unknown             NaN   
17828                      sale associate  sale marketing             NaN   
17829                   android developer         unknown             NaN   
17830                       payroll clerk         unknown             NaN   
17831                     furniture mover         unknown             NaN   

                                                 Profile  \
13092  eme inte

c:\Users\shi.__.ling\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
c:\Users\shi.__.ling\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [24]:
# conduct oversampling since data is imbalanced
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(df.drop(columns=['Fraudulent']), df['Fraudulent'])
df_oresampled = pd.concat([X_resampled, y_resampled], axis=1)
print(df_oresampled)

                                     Job Title   Department Range_of_Salary  \
0                             marketing intern    marketing             NaN   
1      customer service cloud video production      success             NaN   
2        commissioning machinery assistant cma      unknown             NaN   
3              account executive washington dc         sale             NaN   
4                          bill review manager      unknown             NaN   
...                                        ...          ...             ...   
34023                              hse manager      unknown             NaN   
34024                      final expense agent      unknown             NaN   
34025       nursing therapy position home care      unknown             NaN   
34026       home based typist data entry clerk      unknown             NaN   
34027         product development engineer plc  engineering    120000180000   

                                                 Pr

c:\Users\shi.__.ling\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
c:\Users\shi.__.ling\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


In [ ]:
## save as csv
df.to_csv('../Data/Processed/processed_df.csv', index=False)